# 
 Virtual Assistant Behavior Tree using py_trees

In [3]:
!pip install py_trees
import py_trees

# Condition Node: Checks if the calendar is free
class IsCalendarFree(py_trees.behaviour.Behaviour):
    def __init__(self, calendar_free):
        super().__init__(name="Is Calendar Free?")
        self.calendar_free = calendar_free

    def update(self):
        return py_trees.common.Status.SUCCESS if self.calendar_free else py_trees.common.Status.FAILURE

# Condition Node: Checks if rescheduling is allowed
class IsRescheduleAllowed(py_trees.behaviour.Behaviour):
    def __init__(self, reschedule_allowed):
        super().__init__(name="Is Reschedule Allowed?")
        self.reschedule_allowed = reschedule_allowed

    def update(self):
        return py_trees.common.Status.SUCCESS if self.reschedule_allowed else py_trees.common.Status.FAILURE

# Action Node: Schedule the meeting 
class ScheduleMeeting(py_trees.behaviour.Behaviour):
    def __init__(self):
        super().__init__(name="Schedule Meeting")

    def update(self):
        print("Meeting scheduled successfully.")
        return py_trees.common.Status.SUCCESS

# Action Node: Suggest rescheduling
class SuggestReschedule(py_trees.behaviour.Behaviour):
    def __init__(self):
        super().__init__(name="Suggest Reschedule")

    def update(self):
        print("Suggesting reschedule to user.")
        return py_trees.common.Status.SUCCESS

# Action Node: Decline the meeting request
class DeclineRequest(py_trees.behaviour.Behaviour):
    def __init__(self):
        super().__init__(name="Decline Request")

    def update(self):
        print("Cannot schedule meeting. No options available.")
        return py_trees.common.Status.SUCCESS

# Constructs the full behavior tree
def create_tree(calendar_free, reschedule_allowed):
    root = py_trees.composites.Selector(name="Handle Meeting Request", memory=False)

    schedule_sequence = py_trees.composites.Sequence(name="Try Scheduling", memory=False)
    schedule_sequence.add_children([
        IsCalendarFree(calendar_free),
        ScheduleMeeting()
    ])

    reschedule_sequence = py_trees.composites.Sequence(name="Try Rescheduling", memory=False)
    reschedule_sequence.add_children([
        IsRescheduleAllowed(reschedule_allowed),
        SuggestReschedule()
    ])

    root.add_children([
        schedule_sequence,
        reschedule_sequence,
        DeclineRequest()
    ])

    return root

# Run the behavior tree with test cases
print("Test Case: Calendar Free")
tree = py_trees.trees.BehaviourTree(root=create_tree(calendar_free=True, reschedule_allowed=False))
tree.tick()

print("\n Test Case: Calendar Busy but Reschedule Allowed")
tree = py_trees.trees.BehaviourTree(root=create_tree(calendar_free=False, reschedule_allowed=True))
tree.tick()

print("\n Test Case: Calendar Busy and Reschedule Not Allowed")
tree = py_trees.trees.BehaviourTree(root=create_tree(calendar_free=False, reschedule_allowed=False))
tree.tick()

print("\n Test Case: Calendar Free + Reschedule Allowed")
tree = py_trees.trees.BehaviourTree(root=create_tree(calendar_free=True, reschedule_allowed=True))
tree.tick()

print("\n Test Case: Calendar Not Free + Reschedule Not Allowed")
tree = py_trees.trees.BehaviourTree(root=create_tree(calendar_free=False, reschedule_allowed=False))
tree.tick()


Test Case: Calendar Free
Meeting scheduled successfully.

 Test Case: Calendar Busy but Reschedule Allowed
Suggesting reschedule to user.

 Test Case: Calendar Busy and Reschedule Not Allowed
Cannot schedule meeting. No options available.

 Test Case: Calendar Free + Reschedule Allowed
Meeting scheduled successfully.

 Test Case: Calendar Not Free + Reschedule Not Allowed
Cannot schedule meeting. No options available.
